In [5]:
%load_ext autoreload
%autoreload 2
from nbs_uncertainty.readers.bathymetry import load_file
from nbs_uncertainty.processors.bathy_processors import BathyProcessor
from nbs_uncertainty.estimators import uncertaintyEstimators

from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# define bathymetry filename
# directory location defined in config.ini file under config folder
filename = "BlueTopo.tiff"

# Read filename and load bathy data
bathy_data = load_file(filename)
print(bathy_data)

Type: RasterBathymetry
 Filename: BlueTopo.tiff
 Location: ../data/raster
 Resolution: 4.0
 No Data Value: 1000000.0
 Min\Max value: [(np.float32(-106.318), np.float32(-35.669))]
 Data Shape: (2035, 1493)


Warning 1: BlueTopo.tiff: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes


In [7]:
# define processing parameters
linespacing = 32
max_multiple = 4
current_multiple = 1
settings = {'linespacing': linespacing,
            'max_multiple': max_multiple,
            'current_multiple': current_multiple}



In [ ]:
# Next steps:
# 1. Compute interpolated surface?
# 2. Compute residual surface (and compute interpolated surface inside)
# 3.

In [11]:
residual = BathyProcessor.compute_residual(bathy_data, settings)

In [12]:
residual.shape

(2035, 1493)

In [13]:
bathy_data.data.shape

(2035, 1493)

In [ ]:
# estimator = EstimatorSelector.create_estimator(bathy_data)

In [ ]:
# estimator
# raster_estimate = estimate_surface(bathy_data)

In [ ]:
# estimator.set_method("diff_max")
# estimator.set_subsampling_method("across")
# estimator.set_max_multiple(1)
# estimator.set_current_multiple(1)
# estimator.set_linespacing_m(32)


In [ ]:
# estimated_surface_across = estimator.estimate_surface()